- Goal
    - Create synthetic data using IDM
    - Learn parameters of that data using particle filtering
- Learning
    - Need at least 2 vehicles so that there is a neighbor in the front
    - Otherwise src/1d/driver/lane_follower_driver errors
    - That is why the `AutomotiveDrivingModels/doc/1DMobius` stuff does 
    not work with 1 car only (needs at least 2 cars)
    - The tutorial does not work with `gen_straight_roadway` because that
    generates a roadway of type AutomotiveDrivingModels.roadway as opposed to
    AutomotiveDrivingModels.StraightRoadway

In [1]:
using AutomotiveDrivingModels
using AutoViz
using Reel
using Interact
using StatsBase # For random particle generation

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
# 1 lane, 1000 m roadway
# roadway = gen_straight_roadway(1,1000.0) # Does not work with the below because AutomotiveDrivingModels.Roadway
roadway = StraightRoadway(1000.0); # AutomotiveDrivingModels.StraightRoadway type

In [ ]:
typeof(roadway)

In [3]:
scene = Scene1D()
push!(scene, Entity(State1D(10.0,  8.0), VehicleDef(), 1))
push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, pos=VecE2(veh.state.s-0.7, 3)) for veh in scene]
# render(scene, roadway, overlays, cam=cam, canvas_height=100)

2-element Array{AutoViz.TextOverlay,1}:
 AutoViz.TextOverlay
  text: Array{String}((1,))
  color: ColorTypes.RGB{FixedPointNumbers.Normed{UInt8,8}}
  font_size: Int64 10
  pos: Vec.VecE2
  line_spacing: Float64 1.5
  incameraframe: Bool true

 AutoViz.TextOverlay
  text: Array{String}((1,))
  color: ColorTypes.RGB{FixedPointNumbers.Normed{UInt8,8}}
  font_size: Int64 10
  pos: Vec.VecE2
  line_spacing: Float64 1.5
  incameraframe: Bool true


In [34]:
# Set of num_p particles in the range p_start_val to p_end_val
# Each particle represents a desired velocity that parametrizes
# the IDM model
p_start_val = 10.0
p_end_val = 30.0
num_p = 50
p_vdes = sample(p_start_val:p_end_val, num_p, replace = true);

In [39]:
p_vdes[8]

13.0

In [ ]:
# Hallucinate a step using all the particles

In [ ]:
# Compare every particles hallucination to
# the master trajectory (what actually happened)
# and assign a weight

In [4]:
models = Dict{Int, LaneFollowingDriver}()
models[1] = IntelligentDriverModel(v_des=12.0) # always produce zero acceleration
models[2] = IntelligentDriverModel(v_des=20.0) # default IDM with a desired speed of 12 m/s

nticks = 100
timestep = 0.1
rec = QueueRecord(Vehicle1D, nticks+1, timestep)
simulate!(LaneFollowingAccel, rec, scene, roadway, models, nticks)

QueueRecord(nframes=101)

In [21]:
fieldnames(rec.frames[1].entities[1].state)

2-element Array{Symbol,1}:
 :s
 :v

In [25]:
fieldnames(IntelligentDriverModel)

10-element Array{Symbol,1}:
 :a    
 :σ    
 :k_spd
 :δ    
 :T    
 :v_des
 :s_min
 :a_max
 :d_cmf
 :d_max

In [11]:
# From djp chmm bitbucket. src/toy_driving_data.jl
n_cars = scene.n
n_ticks = nticks
X = Array{Float64}(n_cars, n_ticks, 3)

for t in 1:n_ticks
    f = rec.frames[n_ticks - t + 1]
    for c in 1:scene.n
        s = f.entities[c].state
        X[c, t, 1] = s.s #position
        X[c, t, 2] = s.v #speed
    end
end

In [23]:
size(X)

(2, 100, 3)

In [ ]:
using Interact
@manipulate for frame_index in 1 : nframes(rec)
    render(rec[frame_index-nframes(rec)], roadway, cam=cam, canvas_height=100)
end

# 2D experimentation

In [ ]:
roadway = gen_straight_roadway(2,1000.0);

In [ ]:
scene = Scene1D()
push!(scene, Entity(State1D(10.0,  8.0), VehicleDef(), 1))
push!(scene, Entity(State1D(50.0, 12.5), VehicleDef(), 2))

cam = StaticCamera(VecE2(100.0,0.0), 4.75)
overlays = [TextOverlay(text=["$(veh.id)"], incameraframe=true, pos=VecE2(veh.state.s-0.7, 3)) for veh in scene]
render(scene, roadway, overlays, cam=cam, canvas_height=100)

In [ ]:
models = Dict{Int64, DriverModel}()
models[1] = Tim2DDriver(0.1) # always produce zero acceleration
models[2] = Tim2DDriver(0.1) # default IDM with a desired speed of 12 m/s

nticks = 100
timestep = 0.1
rec = QueueRecord(Vehicle1D, nticks+1, timestep)
simulate!(rec, scene, roadway, models, nticks)